In [0]:
# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [0]:
#Cuartiles
from pyspark.sql import functions as F

def categorize_by_quintiles(df, column_names, filter_condition, quintile_probabilities=[0.25, 0.5, 0.75, 1.0], relative_error=0.01):
    for column in column_names:
        # Filtrar y calcular los quintiles
        quintiles = df.filter(filter_condition).approxQuantile(column, quintile_probabilities, relative_error)
        Minimo = df.agg({column: "min"}).collect()[0][0]
        
        # Crear la nueva columna categorizada
        df = df.withColumn(
            f'{column}_Q',
            F.when(F.col(column) <= quintiles[0], f"1. {format(Minimo, '.3f')} - {format(quintiles[0], '.3f')}")
             .when(F.col(column) <= quintiles[1], f"2. {format(quintiles[0], '.3f')} - {format(quintiles[1], '.3f')}")
             .when(F.col(column) <= quintiles[2], f"3. {format(quintiles[1], '.3f')} - {format(quintiles[2], '.3f')}")
             .when(F.col(column) <= quintiles[3], f"4. {format(quintiles[2], '.3f')} - {format(quintiles[3], '.3f')}")
             .otherwise("98. Missing")
        )
    return df

In [0]:
# Configuraciones iniciales
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring

base_seguimiento = spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_gahi")
base_seguimiento = base_seguimiento.withColumn("CODMES", col("CODMES").cast("integer"))
base_seguimiento = base_seguimiento.withColumn("DEF5", when(col("DISTANCIA_DEF24") <= 5, lit(1)).otherwise(lit(0))) 

base_seguimiento = base_seguimiento.withColumn(
    "INGRESO_CONY_TIT",
    coalesce(col("INGRESO_SOL_CONY"), expr("0")) + coalesce(col("INGRESO_RBM"), expr("0"))
)
base_seguimiento = base_seguimiento.withColumn("PD_GAHI_PRELIMINAR", expr("1 / (1 + EXP(-XB_GAHI_PRELIMINAR))"))
base_seguimiento = base_seguimiento.withColumn("mes", col("codmes").cast("string"))
base_seguimiento = base_seguimiento.withColumn("codyear", substring(col("mes"), 1, 4))

base_seguimiento = base_seguimiento.withColumn(
    "RAN_PLAZO",
    F.when(F.col("RAN_PLAZO") == "HASTA 10", "1  HASTA 10")
     .when(F.col("RAN_PLAZO") == "<10 - 15]", "2 <10 - 15]")
     .when(F.col("RAN_PLAZO") == "<15 - 20]", "3 <15 - 20]")
     .when(F.col("RAN_PLAZO") == "MAS DE 20", "4 MAS DE 20")
     .otherwise(F.col("RAN_PLAZO"))
)
base_seguimiento.select("RAN_PLAZO").show(10)

base_seguimiento = base_seguimiento.withColumn(
    "PD_RBM", 
    when(col("codmes") <= 202307, col("PD_ESTADOS")).otherwise(col("CSOL_PD_RBM"))
)

In [0]:
pd_vig='PD_GAHI' 
pd_cal='PD_GAHI'
pd_rbm='PD_RBM'
pd_trc='XB_GAHI_PRELIMINAR'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202305
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Garantia Hipotecaria'

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore_v01(
  nombre_modelo         = 'Personas Modelo Applicant Garantia Hipotecaria',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Garantia Hipotecaria',
  mes_seguimiento       = '202504',
  base_monitoreo        = base_seguimiento,
  pd1                   = pd_cal,
  pd2                   = pd_vig,
  pd3                   = pd_rbm,
  monto_credito         = columna_monto,
  query_univ            = "DEF_120_BLOQ_REF_0=0 AND PD_GAHI IS NOT NULL",
  bandas                = tipo_banda,
  codmes_default        = codmes_default,
  meses_ventana_target  = 24,
  meses_atraso_target   = 4
)

In [0]:
Obj_Seg_cal, rc, rm, fc, fm = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202101",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD18', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "np.where( (calxcuentas['CODMES'] == 202302) | ( calxcuentas['CODMES'] == 202303)| ( calxcuentas['CODMES'] == 202304), (met1_xc) +0.005, (met1_xc) )", #All_proy met1_xc met2_xc met3_xc
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det1_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(met2_xm == 0, met2_xm + 0.005, met2_xm)", #All_proy met1_xm met2_xm met3_xm
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det1_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  #fact_to_proy_xc    = [fc_af, fc_con],
  #fact_to_proy_xm    = [fm_af, fm_con],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.15,
  eje_y_xm           = 0.15,
  dim_grafico        = (25, 6.5),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =20,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rm

In [0]:
# Variables a categoricas
columnas = ["PD_GAHI"]
condicion_filtro = "CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0"
Obj_Seg.base_monitoreo = categorize_by_quintiles(Obj_Seg.base_monitoreo, columnas, condicion_filtro)

In [0]:
driver = 'PD_GAHI_Q'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202101,
  codmes_fin1        = 202212,

  # Periodo reciente
  codmes_inicio2     = 202301,
  codmes_fin2        = 202410,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (22.5, 6),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

In [0]:
Obj_SegDis, evoldis_gini, evoldis_ks = Obj_Seg.EvolDis(
  query_filtro = "CODMES>=201901 AND DEF_120_BLOQ_REF_0 = 0",
  pd_dis       = pd_cal,
  codmes_ini   = 202201,
  codmes_fin   = 202312,
  amplitud     = 24,
  disc_target= [24, 21,18],
  umbral_sup   = 0.5,
  umbral_inf   = 0.4,
  moviles      = True,
  etiquetas= True,
  tamaño_etiqueta=20,
  dim_grafico  = (23, 6),
  pos_leyenda  = (0.5, -0.3)
)

In [0]:
Obj_Seg_dri, rc_dr, rm_dri = Obj_Seg.CalDri(
  driver             = 'codyear',
  query_uso          = f"(CODMES BETWEEN 202101 AND 202305) AND CODMES NOT IN (202201,202208) AND codyear NOT LIKE  '%Missing%' ",
  pd_seg             = pd_cal,
  rd_aux             = 'RD6',
  pd_comparacion     = True,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 0,
  cast_int           = True, 
  titulo             = f'Vista Anual',
  dim_grafico        = (25, 6),
  etiquetas          = True,
  pos_etiquetas_xc   = [10, 10, 10, 10, 10, 10, 10, 10],
  pos_etiquetas_xm   = [10, 10, 10, 10, 10, 10, 10, 10],
  pos_leyenda        = (0.5, -0.2),
  punt_mora          = 100,
  iv                 = False
) 

In [0]:
rm_dri

In [0]:
rc_dr

In [0]:
Obj_Seg_dri, rc_dr, rm_dri = Obj_Seg.CalDri(
  driver             = 'PD_GAHI_Q',
  query_uso          = f"(CODMES BETWEEN 202101 AND 202304) AND CODMES NOT IN (202201,202208) AND PD_GAHI_Q NOT LIKE '%Missing%' ",
  pd_seg             = pd_cal,
  rd_aux             = 'RD6',
  pd_comparacion     = True,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 0,
  cast_int           = True, 
  titulo             = f'Rango de PD',
  dim_grafico        = (25, 6),
  etiquetas          = True,
  pos_etiquetas_xc   = [10, 10, 10, 10, 10, 10, 10, 10],
  pos_etiquetas_xm   = [10, 10, 10, 10, 10, 10, 10, 10],
  pos_leyenda        = (0.5, -0.2),
  punt_mora          = 100,
  iv                 = False
) 